In [1]:
import names
import re
import functions.markovgen as markovgen
import random
import uuid

from datetime import datetime
from time import time
from functions.get_book import *

In [2]:
def random_date(timex=False):
    random.seed(random.random())
    d = random.randint(1, int(time()))

    if timex:
        return datetime.fromtimestamp(d).strftime('%Y-%m-%d %H:%M:%S')
    return datetime.fromtimestamp(d).strftime('%Y-%m-%d')

# Users

In [13]:
regex = re.compile('[^a-zA-Z ]')

file_ = open('descripton_text/text.txt')
results_ = open('./mocks/users.txt', 'w')

markov = markovgen.Markov(file_)
results_.write(r'INSERT INTO "user"(email, name, nickname, description, password, birthday, photo, active, confirmation)')
results_.write('\n')
results_.write('VALUES ')
for i in range(4, 100):
    name = names.get_full_name()

    person = [name.lower().replace(' ', '.') + '@teste.com',
              name,
              name.split(' ')[0],
              regex.sub('', markov.generate_markov_text(5).capitalize()),
              '$2b$12$oVdwTmriZq4StlbbFXIrlO9WxXbwg5TRnv6xIiRBh7OQfdnTb12nu',
              random_date(),
              'https://clipground.com/images/clipart-profile-6.jpg',
              True,
              str(uuid.uuid4())]

    results_.write('(')
    results_.write(',\n'.join([f"'{w}'"for w in person]))
    results_.write('), \n')

file_.close()
results_.close()

# Books

In [4]:
words = ('cara', 'menina', 'menino', 'dragão', 'bruxa', 'água', 'tempo', 'chuva', 'terra', 'choro', 'brasil', 'pedra', 'nome')

aux, id, i = 0, 1, 0
results_ = open('./mocks/books.txt', 'w')

results_.write(r'INSERT INTO book(identifier)')
results_.write('\n')
results_.write('VALUES ')
list_of_books = []
while id < 1000:
    while True:
        books = get_by_term(words[aux], i)
        if 'items' not in books or id > 1000:
            break
        for book in books['items']:
            if book['id'] not in list_of_books:
                results_.write(f"({id}, '{book['id']}'),\n")
                print(id, end='\r')
                id += 1
                list_of_books.append(book['id'])
        i += 1
    aux += 1
    i = 0

    if aux > len(words) - 1:
        break

results_.close()

# User Books e Rate e Likes

In [5]:
regex = re.compile('[^a-zA-Z ]')

id = 1
id_aux = 1
id_aux_2 = 1

results_ = open('./mocks/user_book.txt', 'w')
results_rate = open('./mocks/rate.txt', 'w')
results_likes = open('./mocks/likes_rates.txt', 'w')

file_ = open('descripton_text/biblia.txt', encoding='utf-8')
markov = markovgen.Markov(file_)

results_.write(r'INSERT INTO user_book(fk_book, fk_user, fk_status, date)')
results_.write('\n')
results_.write('VALUES ')

results_rate.write(r' INSERT INTO rate(text, date, rate, likes, fk_user, fk_book)')
results_rate.write('\n')
results_rate.write('VALUES ')

results_likes.write(r' INSERT INTO "like"(fk_comment, fk_rate, fk_user) ')
results_likes.write('\n')
results_likes.write('VALUES ')

all_combinations = []
while id < 1500:
    combination = [random.randint(1, 1000), random.randint(1,99)]
    #Adiciona UserBook
    if combination not in all_combinations:
        status = random.randint(1, 3)
        date = random_date(True)
        results_.write(f"({combination[0]}, {combination[1]}, {status}, '{date}'),\n")
        all_combinations.append(combination)
        # Se tiver for Lido
        if status == 2:
            likes = random.randint(0, 40)
            text = markov.generate_markov_text(random.randint(50, 200)).replace("'", "").replace(':', '').capitalize()
            results_rate.write(f"({id_aux}, '{text}', '{date}', {random.randint(1, 5)}, {likes}, {combination[1]}, {combination[0]}),\n")

            #Se tiver likes
            if likes > 1:
                likes_list = []
                nlikes = 0
                while nlikes < likes:
                    who_likes = random.randint(1, 99)
                    if who_likes not in likes_list:
                        results_likes.write(f"({id_aux_2}, NULL, {id_aux}, {who_likes}),\n")
                        id_aux_2 += 1
                        nlikes += 1
                        likes_list.append(who_likes)
            id_aux += 1
        id += 1


file_.close()
results_.close()
results_rate.close()
results_likes.close()

# User Genre

In [9]:
id = 0

results_ = open('./mocks/user_genre.txt', 'w')

results_.write(r'INSERT INTO user_genre(fk_genre, fk_user)')
results_.write('\n')
results_.write('VALUES ')

all_combinations = []
while id < 700:
    combination = [random.randint(1, 10), random.randint(1, 99)]
    if combination not in all_combinations:
        results_.write(f"({combination[0]}, {combination[1]}),\n")
        id += 1
        all_combinations.append(combination)
results_.close()

# Friends

In [10]:
id = 0

results_ = open('./mocks/friend.txt', 'w')


results_.write(r'INSERT INTO friend(pending, ignored, fk_origin, fk_destiny)')
results_.write('\n')
results_.write('VALUES ')
all_combinations = []
while id < 1500:
    combination = [random.randint(1, 99), random.randint(1, 99)]
    if combination not in all_combinations \
            and combination[0] != combination[1]\
            and combination[::-1] not in all_combinations:
        results_.write(f"({bool(random.getrandbits(1))}, "
                       f"{bool(random.getrandbits(1))}, "
                       f"{combination[0]}, "
                       f"{combination[1]}),\n")
        id += 1
        all_combinations.append(combination)

results_.close()

# Comments

In [12]:
file_ = open('descripton_text/biblia.txt', encoding='utf-8')
results_ = open('./mocks/comments.txt', 'w')
results_likes = open('./mocks/likes_comments.txt', 'w')

markov = markovgen.Markov(file_)
id = 1

results_.write(r'INSERT INTO "comment"(date, text, likes, fk_user, fk_rate)')
results_.write('\n')
results_.write('VALUES ')

results_likes.write(r'INSERT INTO "like"(fk_comment, fk_rate, fk_user) ')
results_likes.write('\n')
results_likes.write('VALUES ')

all_combinations = []
while id < 2500:
    combination = [random.randint(1, 99), random.randint(1, id_aux - 1)]
    if combination not in all_combinations:
        likes = random.randint(0, 20)
        text = markov.generate_markov_text(random.randint(1, 30)).replace("'", "").replace(':', '').capitalize()
        results_.write(f"({id}, '{random_date(True)}', '{text}', {likes}, {combination[0]}, {combination[1]}),\n")
        id += 1
        all_combinations.append(combination)

        if likes > 1:
            likes_list = []
            nlikes = 0
            while nlikes < likes:
                who_likes = random.randint(1, 99)
                if who_likes not in likes_list:
                    results_likes.write(f"({id_aux_2}, {id}, NULL, {who_likes}),\n")
                    id_aux_2 += 1
                    nlikes += 1
                    likes_list.append(who_likes)


results_.close()
results_likes.close()
file_.close

<function TextIOWrapper.close()>